# Imports

In [14]:
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import datetime as dt
import mplfinance as mpf

# Function that Saves Stock Data to CSV

In [18]:
def save_to_csv_from_yahoo(ticker, sdate, edate):
    yf.pdr_override()

    df = web.DataReader(ticker, "yahoo", start=sdate, end=edate)

    df.to_csv("/Users/jwiegand/Downloads/" + ticker + ".csv")
    return df

# Function that Returns a Dataframe from a CSV

# Add Daily Return to Dataframe

# Returns Total Return over Time

# Matplotlib Finance

# Simpole Price Plot

# Test Functions

In [19]:
save_to_csv_from_yahoo("AMZN", "2020-1-1", "2023-12-31")

TypeError: download() got multiple values for argument 'start'